# CaseStudy 4 - Neural Network and Hyper Parameter Tuning

You are provided with a csv file (magic_gamma_telescope04.csv) dataset which
contains telescopic experiment data.
Using the features given in the dataset, you need to
classify the outcome - CLASS.
1. Prepare a model to classify "**class**" using neural networks combined with any HP parameter tuning.
2. Expected: f1 score = 70% and accuracy = 75%.

You can make use of any preprocessing activities if needed.
No external data shall be added for increasing evaluation metric values

In [2]:
!pip install scikeras 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Loading Directories
import tensorflow as tf
from tensorflow import keras
import scikeras 
# Loading Supporting Directories
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [5]:
# Importing dataset
df = pd.read_csv('/content/magic_gamma_telescope04_.csv')
df.head(10)

,flength,fwidth,fsize,fconc,fconc1,fsym,fm3long,fm3trans,falpha,dist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g
5,51.6240,21.1502,2.9085,0.2420,0.1340,50.8761,43.1887,9.8145,3.6130,238.0980,g
6,48.2468,17.3565,3.0332,0.2529,0.1515,8.5730,38.0957,10.5868,4.7920,219.0870,g
7,26.7897,13.7595,2.5521,0.4236,0.2174,29.6339,20.4560,-2.9292,0.8120,237.1340,g
8,96.2327,46.5165,4.1540,0.0779,0.0390,110.3550,85.0486,43.1844,4.8540,248.2260,g
9,46.7619,15.1993,2.5786,0.3377,0.1913,24.7548,43.8771,-6.6812,7.8750,102.2510,g


### PREPROCESSING

In [4]:
df.describe()

,flength,fwidth,fsize,fconc,fconc1,fsym,fm3long,fm3trans,falpha,dist
count,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000
mean,53.250154,22.180966,2.825017,0.380327,0.214657,-4.331745,10.545545,0.249726,27.645707,193.818026
std,42.364855,18.346056,0.472599,0.182813,0.110511,59.206062,51.000118,20.827439,26.103621,74.731787
min,4.283500,0.000000,1.941300,0.013100,0.000300,-457.916100,-331.780000,-205.894700,0.000000,1.282600
25%,24.336000,11.863800,2.477100,0.235800,0.128475,-20.586550,-12.842775,-10.849375,5.547925,142.492250
50%,37.147700,17.139900,2.739600,0.354150,0.196500,4.013050,15.314100,0.666200,17.679500,191.851450
75%,70.122175,24.739475,3.101600,0.503700,0.285225,24.063700,35.837800,10.946425,45.883550,240.563825
max,334.177000,256.382000,5.323300,0.893000,0.675200,575.240700,238.321000,179.851000,90.000000,495.561000


In [5]:
df.describe(include='object')

,class
count,19020
unique,2
top,g
freq,12332


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19020 entries, 0 to 19019
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   flength   19020 non-null  float64
 1   fwidth    19020 non-null  float64
 2   fsize     19020 non-null  float64
 3   fconc     19020 non-null  float64
 4   fconc1    19020 non-null  float64
 5   fsym      19020 non-null  float64
 6   fm3long   19020 non-null  float64
 7   fm3trans  19020 non-null  float64
 8   falpha    19020 non-null  float64
 9   dist      19020 non-null  float64
 10  class     19020 non-null  object 
dtypes: float64(10), object(1)
memory usage: 1.6+ MB


In [7]:
# missing values check
df.isna().sum()

flength     0
fwidth      0
fsize       0
fconc       0
fconc1      0
fsym        0
fm3long     0
fm3trans    0
falpha      0
dist        0
class       0
dtype: int64

In [6]:
# defining data to input and output
X = df.drop('class', axis=1)
y = df['class']

In [7]:
# Scaling the input using MIN MAX
from sklearn.preprocessing import MinMaxScaler
a = MinMaxScaler()
a.fit(X)
X_standardized = a.transform(X)
X = pd.DataFrame(X_standardized)

In [8]:
# Encoding the output
from sklearn.preprocessing import LabelEncoder
# creating instance
le = LabelEncoder()
# fitting
y_ = le.fit_transform(y)

ye = pd.DataFrame(y_)

In [9]:
# importing train test split
from sklearn.model_selection import train_test_split
# Splitting data into train and test data
X_train, X_test, y_train, y_test = train_test_split(X,ye, test_size=0.3, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((13314, 10), (5706, 10), (13314, 1), (5706, 1))

### MODEL BUILDING

In [10]:
# Importing accuracy and f1 score
from sklearn.metrics import accuracy_score, f1_score

Building an initial ANN model randomly to assess performance. Using 2 hidden layers.

In [11]:
# importing for compiler and optimizer
from keras.losses import BinaryCrossentropy
# Creating the model
model = keras.Sequential()

model.add(keras.layers.Dense(10, ))
model.add(keras.layers.Dense(128, activation='relu'))
#Output layer
model.add(keras.layers.Dense(1,activation='sigmoid'))

# Compiling Model
model.compile(optimizer = tf.keras.optimizers.SGD(lr = 0.001, momentum=0.2),
                loss = BinaryCrossentropy(from_logits=True),
                metrics = ['accuracy'])

In [12]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=5)

In [13]:
# fitting the model
model.fit(X_train, y_train, batch_size=12, epochs=100, 
          validation_data=(X_test, y_test), callbacks=[early_stop])

Epoch 1/100
1110/1110 [==============================] - 7s 5ms/step - loss: 0.6557 - accuracy: 0.6377 - val_loss: 0.6304 - val_accuracy: 0.6493
Epoch 2/100
1110/1110 [==============================] - 3s 3ms/step - loss: 0.6204 - accuracy: 0.6480 - val_loss: 0.6098 - val_accuracy: 0.6493
Epoch 3/100
1110/1110 [==============================] - 2s 2ms/step - loss: 0.6033 - accuracy: 0.6491 - val_loss: 0.5936 - val_accuracy: 0.6555
Epoch 4/100
1110/1110 [==============================] - 2s 2ms/step - loss: 0.5877 - accuracy: 0.6654 - val_loss: 0.5776 - val_accuracy: 0.6849
Epoch 5/100
1110/1110 [==============================] - 2s 2ms/step - loss: 0.5719 - accuracy: 0.6979 - val_loss: 0.5614 - val_accuracy: 0.7175
Epoch 6/100
1110/1110 [==============================] - 2s 2ms/step - loss: 0.5562 - accuracy: 0.7219 - val_loss: 0.5455 - val_accuracy: 0.7385
Epoch 7/100
1110/1110 [==============================] - 2s 2ms/step - loss: 0.5413 - accuracy: 0.7342 - val_loss: 0.5305 - val_ac

In [16]:
# predicting model output
y_pred =(model.predict(X_test)>0.5)

In [17]:
print('Accuracy Score: ',accuracy_score(y_test, y_pred))
print('f1 Score      : ',f1_score(y_test, y_pred))

Accuracy Score:  0.8335085874518051
f1 Score      :  0.7347850362925739


Here we have model Accuracy Score of 83.35% and model f1 Score of 73.47%. This meets our requirements. Proceeding to Hypertune the model.

### **HYPERTUNING THE MODEL**

Using GridSearchCV

In [74]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)

#Grid Search CV
from sklearn.model_selection import GridSearchCV

In [126]:
#activation = ['relu', 'sigmoid', 'exponential', LeakyReLU]
optimizer = ['SGD', 'Adam']
#learning_rate = [0.01, 0.02, 0.001]
neurons = [15, 20, 25, 30]
batch_size = [5,10,15]
epochs = [25, 50, 75, 100]

param_grid = dict(model__optimizer=optimizer, 
                  #model__learning_rate=learning_rate,
                  model__epochs =epochs,
                  model__neurons=neurons, 
                 # model__activation=activation, 
                  model__batch_size=batch_size
                  )

In [127]:
# Create model and build Keras Classifier
def ann_fun(optimizer, batch_size, neurons, epochs, ): #learning_rate, activation, 
        opt = 'adam' #
        model = Sequential()
        model.add(Dense(neurons, input_shape=(10,), activation='relu'))
        
        model.add(Dense(1, activation='sigmoid'))
       
        model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
      
        return model

In [128]:
model = KerasClassifier(build_fn=ann_fun, verbose = 0)

In [129]:
grid_ann = GridSearchCV(estimator = model,
                        param_grid = param_grid,
                        scoring= 'accuracy',
                        n_jobs = -1, 
                        cv = 3)

In [130]:
# fitting the model for grid search
grid_ann.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=KerasClassifier(build_fn=<function ann_fun at 0x7fc12fb6a3b0>, verbose=0),
             n_jobs=-1,
             param_grid={'model__batch_size': [5, 10, 15],
                         'model__epochs': [25, 50, 75, 100],
                         'model__neurons': [15, 20, 25, 30],
                         'model__optimizer': ['SGD', 'Adam']},
             scoring='accuracy')

In [131]:
# Finding the best parameters
grid_fit.best_params_

{'model__batch_size': 5,
 'model__epochs': 25,
 'model__neurons': 15,
 'model__optimizer': 'SGD'}

In [138]:
# model creation with best param
model = Sequential()
model.add(Dense(15, input_shape=(10,), activation='relu'))
model.add(Dense(1, activation='sigmoid'))
       
# Compile model
model.compile(loss='binary_crossentropy', optimizer='SGD', metrics=['accuracy'])


In [141]:
# fitting model with best param
model.fit(X_train, y_train, batch_size=5, epochs=25,)
# obtaining output with best param
y_pred = model.predict(X_test)>0.5

Epoch 1/25
2663/2663 [==============================] - 4s 2ms/step - loss: 0.4741 - accuracy: 0.7856
Epoch 2/25
2663/2663 [==============================] - 5s 2ms/step - loss: 0.4653 - accuracy: 0.7890
Epoch 3/25
2663/2663 [==============================] - 4s 2ms/step - loss: 0.4579 - accuracy: 0.7892
Epoch 4/25
2663/2663 [==============================] - 4s 2ms/step - loss: 0.4530 - accuracy: 0.7899
Epoch 5/25
2663/2663 [==============================] - 4s 2ms/step - loss: 0.4477 - accuracy: 0.7917
Epoch 6/25
2663/2663 [==============================] - 5s 2ms/step - loss: 0.4419 - accuracy: 0.7951
Epoch 7/25
2663/2663 [==============================] - 4s 2ms/step - loss: 0.4364 - accuracy: 0.7958
Epoch 8/25
2663/2663 [==============================] - 4s 2ms/step - loss: 0.4302 - accuracy: 0.7992
Epoch 9/25
2663/2663 [==============================] - 6s 2ms/step - loss: 0.4246 - accuracy: 0.8027
Epoch 10/25
2663/2663 [==============================] - 5s 2ms/step - loss: 0.419

In [142]:
print('Accuracy Score: ',accuracy_score(y_test, y_pred))
print('f1 Score      : ',f1_score(y_test, y_pred))


Accuracy Score:  0.8364879074658255
f1 Score      :  0.7375527426160337
